In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fastf1
import plotly.express as px
import fastf1.plotting
import datetime
import requests
from datetime import timedelta

fastf1.Cache.enable_cache('../f1_cache')  

In [75]:
selected_year = 2022
fastf1_session = fastf1.get_session(selected_year, 'Australian', 'Race')
fastf1_session.load(telemetry=False, laps=True, weather=False)

core           INFO 	Loading data for Australian Grand Prix - Race [v2.2.3]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
core           INFO 	Finished loading data for 20 drivers: ['16', '11', '63', '44', '4', '3', '31', '77', '10', '23', '24', '18', '47', '20', '22', '6', '14', '1', '5', '55']


# LapTime vs LapNumber

In [37]:
selected_drivers = ['HAM', 'RIC']
lap_time_number = fastf1_session.laps[['Driver', 'LapTime', 'Compound', 'TyreLife', 'LapNumber']].copy()
lap_time_number = lap_time_number[lap_time_number['Driver'].isin(selected_drivers)]
lap_time_number['LapTime'] = lap_time_number['LapTime'].fillna(timedelta(0))
lap_time_number['Compound'] = lap_time_number['Compound'].fillna("")
lap_time_number['Year'] = selected_year
lap_time_number['TyreLife'] = lap_time_number['TyreLife'].fillna(-1)
lap_time_number['LapTime'] = pd.to_numeric(lap_time_number['LapTime'])/1000000
lap_time_number_cov = lap_time_number.to_dict('records')
final_dict = {}
for data in lap_time_number_cov:
    primary_key = data['Driver']
    data[primary_key] = data.pop('LapTime')

    # color
    driv_lap = fastf1_session.laps.pick_driver(primary_key).pick_fastest()
    color = fastf1.plotting.team_color(driv_lap['Team'])
    data['color'] = color

    if primary_key not in final_dict:
        final_dict[primary_key] = [data]
    else:
        final_dict[primary_key].append(data)

# for driv in final_dict:
#     thelist = final_dict[driv]
#     thelist[:] = [d for d in thelist if d.get(driv) != 0]

# Team Color Setup

In [73]:
team_color_pair = {'VER': '#1a5d57',
 'PER': '#0600ef',
 'RUS': '#005aff',
 'LAT': '#005aff',
 'BOT': '#00d2be',
 'HAM': '#00d2be',
 'GAS': '#2b4562',
 'ALO': '#0090ff',
 'OCO': '#0090ff',
 'SAI': '#dc0000',
 'LEC': '#dc0000',
 'RAI': '#900000',
 'GIO': '#900000',
 'MSC': '#ffffff',
 'TSU': '#2b4562',
 'MAZ': '#ffffff',
 'NOR': '#ff8700',
 'RIC': '#ff8700',
 'VET': '#006f62',
 'HUL': "006f62"
 'STR': '#006f62'}

In [83]:
pairs = {}
selected_drivers = ['VER', 'PER', 'RUS', 'LAT', 'BOT', "HAM", 'GAS', 'ALO', 'OCO', 'SAI', 'LEC', 'RAI', 'GIO', 'MSC', 'TSU', 'MAZ', 'NOR', 'RIC', 'VET', 'STR', 'HUL', 'MAG']
selected_drivers = ['HUL']
for driver in selected_drivers:
    driv_lap = fastf1_session.laps.pick_driver(driver).pick_fastest()
    try:
        print(driv_lap)
        color = fastf1.plotting.team_color(driv_lap['Team'])
    except:
        color = team_color_pair[driver]
    pairs[driver] = color
pairs

Time                 NaN
DriverNumber         NaN
LapTime              NaN
LapNumber            NaN
Stint                NaN
PitOutTime           NaN
PitInTime            NaN
Sector1Time          NaN
Sector2Time          NaN
Sector3Time          NaN
Sector1SessionTime   NaN
Sector2SessionTime   NaN
Sector3SessionTime   NaN
SpeedI1              NaN
SpeedI2              NaN
SpeedFL              NaN
SpeedST              NaN
IsPersonalBest       NaN
Compound             NaN
TyreLife             NaN
FreshTyre            NaN
LapStartTime         NaN
Team                 NaN
Driver               NaN
TrackStatus          NaN
IsAccurate           NaN
dtype: float64


KeyError: 'HUL'

## Speed vs Distance

In [45]:
selected_drivers = ['HAM', 'ALO', 'LAT']
final_dict = {}
for driver in selected_drivers:
    driv_tel = {}
    driv_lap = fastf1_session.laps.pick_driver(driver).pick_fastest()
    color = fastf1.plotting.team_color(driv_lap['Team'])
    driv_tel['Data'] = driv_lap.get_car_data().add_distance()[['Speed', 'Distance']].rename(columns={"Speed": driver}).to_dict('records')
    driv_tel['Color'] = color
    final_dict[driver] = driv_tel

In [73]:
driv_lap = fastf1_session.laps.pick_driver("HAM").pick_fastest()
driv_lap.get_car_data()


,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime
0,2021-09-12 13:06:29.974,11315,313,8,100,False,1,car,0 days 00:00:00.231000,0 days 01:05:28.365000
1,2021-09-12 13:06:30.214,11393,315,8,100,False,1,car,0 days 00:00:00.471000,0 days 01:05:28.605000
2,2021-09-12 13:06:30.454,11481,317,8,100,False,1,car,0 days 00:00:00.711000,0 days 01:05:28.845000
3,2021-09-12 13:06:30.694,11509,318,8,100,False,1,car,0 days 00:00:00.951000,0 days 01:05:29.085000
4,2021-09-12 13:06:30.934,11573,320,8,100,False,1,car,0 days 00:00:01.191000,0 days 01:05:29.325000
...,...,...,...,...,...,...,...,...,...,...
351,2021-09-12 13:07:54.415,10978,302,8,100,False,0,car,0 days 00:01:24.672000,0 days 01:06:52.806000
352,2021-09-12 13:07:54.655,11002,304,8,100,False,0,car,0 days 00:01:24.912000,0 days 01:06:53.046000
353,2021-09-12 13:07:54.895,11134,306,8,100,False,0,car,0 days 00:01:25.152000,0 days 01:06:53.286000
354,2021-09-12 13:07:55.135,11157,309,8,100,False,0,car,0 days 00:01:25.392000,0 days 01:06:53.526000


In [36]:
f1_season = fastf1.get_event_schedule(int(2021))
event_records = f1_season[['RoundNumber', 'EventName']]
round_number = pd.DataFrame(event_records)[event_records['EventName'] == 'Monaco Grand Prix']['RoundNumber']

In [43]:
type(round_number.iloc[0])

numpy.int64

In [21]:
driver_list = [x[:3] for x in requests.get(
            "http://ergast.com/api/f1/{}/{}/drivers".format(2021, round_number))]

In [24]:
requests.get("http://ergast.com/api/f1/{}/{}/drivers".format(2021, round_number)).text

'<html>\r\n  <head>\r\n    <title>Error</title>\r\n  </head>\r\n  <body>\r\n    <h3>Bad Request</h3>\r\n  </body>\r\n</html>\r\n'